In [0]:
!pip install face-detection

In [0]:
from google.colab import drive
drive.mount("drive")

In [0]:
from keras.applications import resnet50
from keras.models import Model
from keras.layers import Dense,Dropout,AveragePooling2D,Flatten
from keras.preprocessing.image import load_img,img_to_array

In [0]:
base_network  = resnet50.ResNet50(input_shape = (224,224,3),weights='imagenet',include_top=False)

In [0]:
classifier_network = base_network.output
classifier_network = AveragePooling2D(pool_size=(7, 7),name = "Average_Pool_Final")(classifier_network)
classifier_network = Flatten(name = "Flatten_Final")(classifier_network)
classifier_network = Dense(128, activation="relu",name = "Dense_Final")(classifier_network)
classifier_network = Dropout(0.5,name = "Dropout_Final")(classifier_network)
classifier_network = Dense(1, activation="sigmoid", name = "Sigmoid_Classifier" )(classifier_network)

mask_classifier = Model(inputs=base_network.input, outputs=classifier_network)

In [0]:
for layer in base_network.layers:
  layer.trainaibale = False

In [0]:
Masked_Path = "drive/My Drive/Social_Distancing/data/with_mask"
UnMasked_Path = "drive/My Drive/Social_Distancing/data/without_mask"

faces = []
labels = []

paths = [UnMasked_Path,Masked_Path]

loaded = 0 

for i in range(2):

  for file_name in os.listdir(paths[i]):

    try:
      
      image = cv2.imread(os.path.join(paths[i],file_name))

      detections = detector.detect(image[:,:,::-1])

      for j in range(len(detections)):
    
        face = image[int(detections[j][1]):int(detections[j][3]),int(detections[j][0]):int(detections[j][2]),::-1]
        face = cv2.resize(face, (224, 224), interpolation=cv2.INTER_NEAREST)
        face = resnet50.preprocess_input(face)
        faces.append(face)
        labels.append(i)

      loaded+=1

      if loaded%100==0:
        print(str(loaded)+" images loaded successfuly !")
    
    except:
      print("Error loading "+str(os.path.join(paths[i],file_name))+" !")

faces = np.array(faces)
labels = np.array(labels)